This package allows you to construct and export mtg into 4 formats : .mtg, .opf, .vtp and .gltf

In [ ]:
import os
import format_io as f_io #trop general
                        #plant archi ... je trouve un nom PlantConvert 
    #pltformat mieux que format

from _const import OPFS

DATA_PATH = OPFS
ext = "opf"

#example opf filenames
simple_plant = "%s%s.%s"%(OPFS, "simple_plant", ext)
coffee = "%s%s.%s"%(OPFS, "coffee", ext)
palm = "%s%s.%s"%(OPFS, "DA1_Average_MAP_90", ext)

Let's first read a simple plant file and visualize it with plantgl. We use an object oriented approach to implement the parser / mtg constructer because we intent to deal with complex data structure.

You can instantiate an input/output object using `format_io.io()` with the file name and then ask the object to parse the file by object method `io.read()`. The mtg will be saved in the attribute `io.g`.

The `read` method will detect the extension of the file name to determin the specific read method. You can also calling those methods without using the input/output object by calling them directly from `format_io.vtk`, `format_io.opf` and `format_io.gltf` directly.

In [ ]:
import openalea.plantgl.all as pgl

In [ ]:
io = f_io.io(file=simple_plant)
io.read()

In [ ]:
io.g.display()

In [ ]:
%gui qt
scene = f_io.geometry.get_scene(io.g)
pgl.Viewer.display(scene)

The meshes are stored as propreties of each node who has associated geometry, the previous function `get_scene` simply extract each geometry property and assemble them into a plantgl scene object. 

The list of reference meshes are stored as a node attribute of the node at scale 0. They are supposed to be stored in a dictionnary with mesh_id as keys and a tuple as values. The tuple contains a triangle mesh (TriangleSet object of plantgl) and a boolean that indicates if tapering should be applied on the mesh (this is made for compatibilitiy with opf and yet not very optimized...)

In [ ]:
prop = io.g.properties()
geo = prop['geometry']
print(geo)
ref_geo = prop["ref_meshes"]
print(ref_geo)

Now we can try another more complex example. To read another `opf` file, you just need to change the `file` attribute of the object `io`. And ask for parsing again using the `read` method.

In [ ]:
io.file = coffee #you can also try with palm
io.read()

In [ ]:
scene = f_io.geometry.get_scene(io.g)
pgl.Viewer.display(scene)

To write the currently loaded mtg into another file format, use the `write` method by providing a new filename. Let's switch back to the simple_plant as it's faster to write into file.

The [gltf](https://www.khronos.org/registry/glTF/specs/2.0/glTF-2.0.html) format is a xml styled format that allows to save a scene graph (a hierarchy of geometric objects to be rendered, we exploit this hierarchy to encode the topology of a mtg).

In [ ]:
io.file = simple_plant
io.read()
io.write("simple_plant.gltf")

In [ ]:
# shell command to print the gltf file
%cat simple_plant.gltf

# Write opf with geometry from a mtg file
You can load a mtg file and write it into opf format. The opf format doesn't require necessarily geometries. When you generate geometry from the mtg file and if you want to output an opf equipped with geoemtry, you should call the method `f_io.opf.writer.apply_scene(g, scene)` to associate a plantgl scene with mtg's vertices.

Inside this function, `scene.todict()` will be applied and we expect that the keys correspond to the vertices id that contains the geometry. By calling `apply_scene`, the property `geometry` and `ref_meshes` will be created and you can write opf with geometry as usual.

We intent to extract reference meshes from scene, but it's difficult to do so in general case. The method `apply_scene` saves each mesh as reference mesh and the transformation is always identity.

In `strawberry/` there is a working example using `openalea.strawberry` which provides a function to generate 3D geometry of a scene. Then we associate the geometry to the mtg and write it into opf.

>format et visu (oawidgets)